## **MEDIAPIPE face mesh**

link: https://google.github.io/mediapipe/solutions/face_mesh

**Configuration Options**

1.  **STATIC_IMAGE_MODE**
*   = False → treats input as a video
MAX_NUM_FACES
2.  **MAX_NUM_FACES** 
* Maximum number of faces to detect
3.   **REFINE_LANDMARKS**
*  tinh chỉnh thêm tọa độ mốc xung quanh mắt và môi
*  xuất ra các mốc bổ sung xung quanh tròng đen
4. **MIN_DETECTION_CONFIDENCE**
* confidence from the *face detection model* for the detection  
5. **MIN_TRACKING_CONFIDENCE**
* confidence from the *landmark-tracking model* for landmarks



**OUTPUTS:**
* detected faceS
  * 1 face = list of landmarks
      * 1 landmark = x, y, z



In [4]:
import cv2
import mediapipe as mp

# utils for drawing on image
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

#face_mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh=mp_face_mesh.FaceMesh
cap = cv2.VideoCapture(0)

In [7]:
import cv2
import mediapipe as mp

# utils for drawing on image
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
#face_mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh
khanh=[]
import math

cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
  while cap.isOpened():
    success, image = cap.read()
    image = cv2.flip(image,1)
    
    w  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float `width`
    h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float `height`
    
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    face_landmarks = results.multi_face_landmarks
    
    if results.multi_face_landmarks is not None:
        for face_landmarks in results.multi_face_landmarks:
            khanh.append(face_landmarks)

   #def function
    def take_landmark(num):
        results = [face_landmarks.landmark[num].x * w, face_landmarks.landmark[num].y * h,face_landmarks.landmark[num].z * w]
        return results
    def take_distance(num1,num2):
        distances = math.sqrt((face_landmarks.landmark[num1].x *w- face_landmarks.landmark[num2].x *w)**2 +(face_landmarks.landmark[num1].y *h- face_landmarks.landmark[num2].y*h)**2+(face_landmarks.landmark[num1].z*w- face_landmarks.landmark[num2].z*w)**2 )
        return distances
    def draw_line(num1,num2):
        color=(255, 0, 0)
        return cv2.line(image,(int(take_landmark(num1)[0]),int(take_landmark(num1)[1])),(int(take_landmark(num2)[0]),int(take_landmark(num2)[1])),color,1)
    def put_text(distance,a1,a2):
        return cv2.putText(image,str(distance),(a1,a2), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1)
           
    #Đường kính mắt: 474,476
    diameter_mm=11.2/take_distance(474,476) 

    # Face Length: 152, 10
    facelength_distance = take_distance(152,10)*diameter_mm

    ## Iris: 473, 468
    pd_distance=take_distance(473,468)*diameter_mm

    ## FOREHEAD: 54, 333
    forehead_distance=take_distance(54,333)*diameter_mm

    # JAWLINE: 58, 433
    jawline_distance=take_distance(58,433)*diameter_mm

    # OUTER EYE: 130, 359
    outereye_distance=take_distance(130,359)*diameter_mm

    #INNER EYE: 112, 362
    innereye_distance=take_distance(112,363)*diameter_mm

    #CHEEKBONE: 227, 454
    cheekbone_distance1=take_distance(227,454)
    cheekbone_distance=take_distance(93,132)+take_distance(132,58)+take_distance(58,172)+take_distance(136,172)+take_distance(136,150)+take_distance(150,149)+take_distance(149,176)+take_distance(176,152)+take_distance(152,337)+take_distance(337,400)+take_distance(400,378)+take_distance(378,379)+take_distance(379,365)+take_distance(365,397)+take_distance(397,361)+take_distance(361,323)
    cheekbone_distance=cheekbone_distance*diameter_mm

    #TEMPLE WIDTH: 127, 372 
    templewidth_distance=take_distance(127,372)

    if results.multi_face_landmarks is not None:
        for face_landmarks in results.multi_face_landmarks:
            #cv2.circle(annotated_image, ( int(forehead_left[0]),int(forehead_left[1]) ), 5, (255,255,0), -2)
            # draw LINE
            draw_line(152,10) ## Face Length: 152, 10
            draw_line(54,333) ## FOREHEAD: 54, 333
            draw_line(473,468) ## Iris: 473, 468
            draw_line(130,359) # OUTER EYE: 130, 359
            draw_line(112,362) #INNER EYE: 112, 362
            draw_line(127,372) #TEMPLE WIDTH: 127, 372 
            # cheeck bone
            draw_line(93,132);draw_line(132,58);draw_line(58,172);draw_line(172,136);draw_line(136,150);draw_line(150,149);draw_line(149,176);draw_line(176,148);draw_line(148,152)
            draw_line(152,377);draw_line(377,400);draw_line(400,378);draw_line(378,379);draw_line(379,365);draw_line(365,397);draw_line(397,361);draw_line(361,323)

            cv2.putText(image,str(facelength_distance),(50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1)
            cv2.putText(image,str(forehead_distance),(50,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1)
            cv2.putText(image,str(jawline_distance),(50,250), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1)
            cv2.putText(image,str(cheekbone_distance),(50,350), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1)
            #cv2.putText(image,str(pd_distance),(50,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1)
            #cv2.putText(image,str(outereye_distance),(50,350), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1)
            #cv2.putText(image,str(innereye_distance),(50,450), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1)
            #cv2.putText(image,str(templewidth_distance),(50,650), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1)
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Face Mesh', image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
cap.release() # release the cap object
cv2.destroyAllWindows() # close all windows